<img src="NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing <b> </font>

<font size="5"> <b>Lab 7: Deep Learning in Earth Observation: Taizhou Change Detection </b> </font>

<br>
<font size="4"> <b> Lichao Mou, German Aerospace Center; Xiaoxiang Zhu, German Aerospace Center & Technical University Munich </b> <br>
</font>

<img src="NotebookAddons/dlr-logo-png-transparent.png" width="170" align="right" border="2"/> <font size="3"> This Lab introduces you to the basic concepts of Deep Learning in Earth Observation. Specifically, it uses Convolutional Recurrent Neural Networks (CRNNs) to perform a multi-temporal change detection on multispectral data collected over Taizhou, China. The images are both 400 × 400 pixels in size and show significant changes mainly related to city expansion, soil change, and varying water areas.
    
We will again use a **Jupyter Notebook** framework implemented within the Amazon Web Services (AWS) cloud to work on this exercise. This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It will introduce the following data analysis concepts:

- How to set up a convolutional recurrent deep network within the Python-based <i>keras/tensorflow</i> environment
- How to use CRNNs to perform change detection on multi-temporal remote sensing data 
</font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>There are no Homework assignments associated with this Notebook </b> </font>
</font>
<br>
<hr>

<hr>
<font face="Calibri" size="5" color="darkred"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>

<hr>
<font face="Calibri" size="5"> <b>0. Importing Relevant Python Packages </b> </font>

<font size="3">Our first step is to <b>import the necessary python libraries into your Jupyter Notebook.</b></font>

In [ ]:
!pip install --user tensorflow

In [ ]:
!pip install --user keras

In [ ]:
%%capture
import os

import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Nadam
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers import Conv2D, Reshape, Activation, Concatenate, GRU, Dense, LSTM, SimpleRNN

import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()

<hr>
<font face="Calibri" size="5"><b>1. Create a working directory for the analysis and change into it:</b></font>

In [ ]:
base_path = "/home/jovyan/notebooks/ASF/GEOS_657_Labs/2019/lab_7_taizhou_data"
asfn.new_directory(base_path)
print(f"Current working directory: {os.getcwd()}")

<hr>
<font face="Calibri" size="5"><b>2. Data Preparation</b></font> 

<font face="Calibri" size="3"><b>load T1 and T2 images, training map, and test map. Save the images (T1.png and T2.png):</b></font> 

In [ ]:
patch_size = 3
num_bands = 6
print('########## load data... ##########')
data = sio.loadmat('DL-data/Taizhou_3x3/TaizhouTm2000_norm.mat')
imgT1 = np.float32(data['imgT1'])
data = sio.loadmat('DL-data/Taizhou_3x3/TaizhouTm2003_norm.mat')
imgT2 = np.float32(data['imgT2'])

data = sio.loadmat('DL-data/Taizhou_3x3/TaizhouTraMapBinary.mat')
tra_map = np.uint8(data['tra_map_binary'])
data = sio.loadmat('DL-data/Taizhou_3x3/TaizhouTestMapBinary.mat')
test_map = np.uint8(data['test_map_binary'])

print('the shape of T1 image is: {}'.format(imgT1.shape))
print('the shape of T2 image is: {}'.format(imgT2.shape))

plt.imshow(imgT1[:, :, [3, 2, 1]])
plt.savefig(f"{base_path}/T1.png", dpi=300)
plt.show()

plt.imshow(imgT2[:, :, [3, 2, 1]])
plt.savefig(f"{base_path}/T2.png", dpi=300)
plt.show()

[rows, cols] = np.nonzero(tra_map)
num_samples = len(rows)
rows = np.reshape(rows, (num_samples, 1))
cols = np.reshape(cols, (num_samples, 1))
temp = np.concatenate((rows, cols), axis=1)
np.random.shuffle(temp)
rows = temp[:, 0].reshape((num_samples,))
cols = temp[:, 1].reshape((num_samples,))

<font face="Calibri" size="3">Create 3x3 patches as training samples according to the training map</font> 
<br><br>
<font face="Calibri" size="3"><b>Create numpy arrays temporarily filled with zeros to hold our 3x3 patches:</b></font> 

In [ ]:
x_tra_t1 = np.float32(
    np.zeros([num_samples, patch_size, patch_size, num_bands]))
x_tra_t2 = np.float32(
    np.zeros([num_samples, patch_size, patch_size, num_bands]))

y_tra = np.uint8(np.zeros([num_samples, ])) # ground truths for training samples

<font face="Calibri" size="3"><b>Populate the zero-filled arrays with appropriate values:</b></font> 

In [ ]:
for i in range(num_samples):
    patch = imgT1[rows[i]-int((patch_size-1)/2): rows[i]+int((patch_size-1)/2)+1,
                  cols[i]-int((patch_size-1)/2): cols[i]+int((patch_size-1)/2)+1, :]
    x_tra_t1[i, :, :, :] = patch
    patch = imgT2[rows[i]-int((patch_size-1)/2): rows[i]+int((patch_size-1)/2)+1,
                  cols[i]-int((patch_size-1)/2): cols[i]+int((patch_size-1)/2)+1, :]
    x_tra_t2[i, :, :, :] = patch
    y_tra[i] = tra_map[rows[i], cols[i]]-1

[rows, cols] = np.nonzero(test_map)
num_samples = len(rows)
rows = np.reshape(rows, (num_samples, 1))
cols = np.reshape(cols, (num_samples, 1))
temp = np.concatenate((rows, cols), axis=1)
np.random.shuffle(temp)
rows = temp[:, 0].reshape((num_samples,))
cols = temp[:, 1].reshape((num_samples,))

<font face="Calibri" size="3"><b>Sample 3x3 patches as test samples according to the test map:</b></font> 

In [ ]:
# test samples from T1 image
x_test_t1 = np.float32(
    np.zeros([num_samples, patch_size, patch_size, num_bands]))
# test samples from T2 image
x_test_t2 = np.float32(
    np.zeros([num_samples, patch_size, patch_size, num_bands]))
# ground truths for test samples
y_test = np.uint8(np.zeros([num_samples, ]))  
for i in range(num_samples):
    patch = imgT1[rows[i]-int((patch_size-1)/2): rows[i]+int((patch_size-1)/2)+1,
                  cols[i]-int((patch_size-1)/2): cols[i]+int((patch_size-1)/2)+1, :]
    x_test_t1[i, :, :, :] = patch
    patch = imgT2[rows[i]-int((patch_size-1)/2): rows[i]+int((patch_size-1)/2)+1,
                  cols[i]-int((patch_size-1)/2): cols[i]+int((patch_size-1)/2)+1, :]
    x_test_t2[i, :, :, :] = patch
    y_test[i] = test_map[rows[i], cols[i]]-1

print('the shape of input tensors on training set is: {}'.format(x_tra_t1.shape))
print('the shape of target tensor on training set is: {}'.format(y_tra.shape))
print('the shape of input tensors on training set is: {}'.format(x_test_t1.shape))
print('the shape of target tensor on training set is: {}'.format(y_test.shape))

<hr>
<font face="Calibri" size="5"> <b>3. Building up the recurrent convolutional network </b> </font> 

<font face="Calibri" size="3"><b>Write a function to build the network:</b></font> 

In [ ]:
def build_network():
    # the T1 branch of the convolutional sub-network
    input1 = Input(shape=(3, 3, 6))
    x1 = Conv2D(filters=32, kernel_size=3, strides=1, padding='valid')(input1)
    x1 = Activation('relu')(x1)
    x1 = Reshape(target_shape=(1, 32))(x1)

    # the T2 branch of the convolutional sub-network
    input2 = Input(shape=(3, 3, 6))
    x2 = Conv2D(filters=32, kernel_size=3, strides=1, padding='valid')(input2)
    x2 = Activation('relu')(x2)
    x2 = Reshape(target_shape=(1, 32))(x2)

    # the recurrent sub-network
    x = Concatenate(axis=1)([x1, x2])
    #x = SimpleRNN(units = 128)(x)
    x = LSTM(units=128)(x)
    #x = GRU(units = 128)(x)
    x = Dense(units=32, activation='relu')(x)
    y = Dense(units=1, activation='sigmoid')(x)

    net = Model(inputs=[input1, input2], outputs=y)

    net.summary()

    return net


<hr>
<font face="Calibri" size="5"> <b>4. Network training </b> </font> 

<font face="Calibri" size="3"><b>Build the network:</b></font> 

In [ ]:
print('########## train the network... ##########')
batch_size = 32
nb_epoch = 200
net = build_network()

<font face="Calibri" size="3"><b>Train the network:</b></font> 

In [ ]:
nadam = Nadam(lr=0.00002)
net.compile(optimizer=nadam, loss='binary_crossentropy', metrics=['accuracy'])
net_info = net.fit([x_tra_t1, x_tra_t2], y_tra, batch_size=batch_size, validation_split=0.1, epochs=nb_epoch)

loss = net_info.history['loss']
loss_val = net_info.history['val_loss']
plt.rcParams.update({'font.size': 18})
fig = plt.figure(figsize=(8, 7))
ax = fig.add_subplot(1, 1, 1)
plt.plot(loss)
plt.plot(loss_val)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.savefig(f"{base_path}/loss.png", bbox_inches='tight', dpi=200)
plt.show()
#sio.savemat('loss_curves.mat', {'loss': loss, 'loss_val': loss_val})

<hr>
<font face="Calibri" size="5"><b>5. Test</b></font> 

<font face="Calibri" size="3"><b>Run the network on the test dataset. Save the change map probability and the change map binary (change_map_probability.png and change_map_binary.png):</b></font> 

In [ ]:
print('########## test... ##########')
# testing on test set
score = net.evaluate([x_test_t1, x_test_t2], y_test)
print(score[1])

print('########## running on the whole image... ##########')
cnt = 0
x_t1 = np.float32(np.zeros([400*400, patch_size, patch_size, num_bands]))
x_t2 = np.float32(np.zeros([400*400, patch_size, patch_size, num_bands]))
print('sampling patches...')
for i in range(1, imgT1.shape[0]-1, 1):
    for j in range(1, imgT1.shape[1]-1, 1):
        patch = imgT1[i-int((patch_size-1)/2): i+int((patch_size-1)/2)+1,
                      j-int((patch_size-1)/2): j+int((patch_size-1)/2)+1, :]
        x_t1[cnt, :, :, :] = patch
        patch = imgT2[i-int((patch_size-1)/2): i+int((patch_size-1)/2)+1,
                      j-int((patch_size-1)/2): j+int((patch_size-1)/2)+1, :]
        x_t2[cnt, :, :, :] = patch
        cnt = cnt + 1
print('sampling done.')
pred = net.predict([x_t1, x_t2])
change_map_prob = np.reshape(pred, (400, 400))
plt.imshow(change_map_prob)
plt.savefig(f"{base_path}/change_map_probability.png", dpi=200)
plt.show()

change_map_binary = np.where(change_map_prob < 0.5, 0, 1)
plt.imshow(change_map_binary)
plt.savefig(f"{base_path}/change_map_binary.png", dpi=200)
plt.show()

<font face="Calibri" size="2"> <i>GEOS 657-Lab7-DeepLearning-TaizhouChangeDetect - Version 1.2.0 - April 2021
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>namespace asf_notebook</li>
    </ul>
    </i>
</font>